### Libraries

In [1]:
import pandas as pd
import re

In [2]:
from ingest.catalog import DatasetCatalog
from ingest.loader import RawDatasetLoader
from ingest.fetch.csv import CsvAdapter
from ingest.fetch.sct import SocrataAdapter
from ingest.fetch.excel import ExcelAdapter
from ingest.fetch.metadata import SocrataMetadata
from utils.clean import normalize_numeric_code
from config import DATA_PATH

In [3]:
## servicios
catalog = DatasetCatalog()
loader = RawDatasetLoader(
    csv_adapter=CsvAdapter(),
    sct_adapter=SocrataAdapter(),
    excel_adapter=ExcelAdapter(),
)

### Info Dataset

In [4]:
ds = catalog.get("ciuu_4ac_dane")
records = list(loader.load(ds))
df_ciiu = pd.DataFrame(records)
df_ciiu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 713 entries, 0 to 712
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   División     713 non-null    object
 1   Grupo        713 non-null    object
 2   Clase        713 non-null    object
 3   Descripción  713 non-null    object
dtypes: object(4)
memory usage: 22.4+ KB


In [5]:
df_ciiu.head(20)

,División,Grupo,Clase,Descripción
0,ESTRUCTURA DETALLADA DE LA CLASIFICACIÓN INDUS...,,,
1,,,,
2,SECCIÓN A,,,"AGRICULTURA, GANADERÍA, CAZA, SILVICULTURA Y P..."
3,01,,,"Agricultura, ganadería, caza y actividades de ..."
4,,011,,Cultivos agrícolas transitorios
5,,,0111,"Cultivo de cereales (excepto arroz), legumbres..."
6,,,0112,Cultivo de arroz
7,,,0113,"Cultivo de hortalizas, raíces y tubérculos"
8,,,0114,Cultivo de tabaco
9,,,0115,Cultivo de plantas textiles


### Cleaning

In [6]:
df_ciiu = df_ciiu.rename(columns={
    "División": "division",
    "Grupo": "grupo",
    "Clase": "clase",
    "Descripción": "description"
})

df_ciiu.grupo = df_ciiu.grupo.apply(lambda x: normalize_numeric_code(x, digits=3))
df_ciiu.clase = df_ciiu.clase.apply(lambda x: normalize_numeric_code(x, digits=4))

In [7]:
division_descs = {}
group_descs = {}
class_descs = {}

for _, row in df_ciiu.iterrows():
    div  = row.division
    grp  = row.grupo
    cls  = row.clase
    desc = row.description

    div_clean = normalize_numeric_code(div, digits=2)
    if div_clean:
        division_descs[div_clean] = desc
    if grp:
        group_descs[grp] = desc
    if cls:
        class_descs[cls] = desc


In [8]:
rows = []
current_section_code = None
current_section_desc = None

for _, row in df_ciiu.iterrows():

    div = row.division
    cls = row.clase
    desc = row.description
    if isinstance(div, str) and div.upper().startswith("SECCIÓN"):
        current_section_code = div.split()[-1]
        current_section_desc = desc
        continue

    if cls:
        ciiu_code = cls
        division_code = ciiu_code[:2]
        group_code    = ciiu_code[:3]

        rows.append({
            "seccion_code": current_section_code,
            "seccion_desc": current_section_desc,

            "division_code": division_code,
            "division_desc": division_descs.get(division_code),

            "group_code": group_code,
            "group_desc": group_descs.get(group_code),

            "ciiu_code": ciiu_code,
            "ciiu_desc": class_descs.get(ciiu_code)
        })
df_ciiu_desc = pd.DataFrame(rows)
df_ciiu_desc = df_ciiu_desc.sort_values(["seccion_code", "division_code", "group_code", "ciiu_code"])
df_ciiu_desc.head()

,seccion_code,seccion_desc,division_code,division_desc,group_code,group_desc,ciiu_code,ciiu_desc
0,A,"AGRICULTURA, GANADERÍA, CAZA, SILVICULTURA Y P...",01,"Agricultura, ganadería, caza y actividades de ...",011,Cultivos agrícolas transitorios,0111,"Cultivo de cereales (excepto arroz), legumbres..."
1,A,"AGRICULTURA, GANADERÍA, CAZA, SILVICULTURA Y P...",01,"Agricultura, ganadería, caza y actividades de ...",011,Cultivos agrícolas transitorios,0112,Cultivo de arroz
2,A,"AGRICULTURA, GANADERÍA, CAZA, SILVICULTURA Y P...",01,"Agricultura, ganadería, caza y actividades de ...",011,Cultivos agrícolas transitorios,0113,"Cultivo de hortalizas, raíces y tubérculos"
3,A,"AGRICULTURA, GANADERÍA, CAZA, SILVICULTURA Y P...",01,"Agricultura, ganadería, caza y actividades de ...",011,Cultivos agrícolas transitorios,0114,Cultivo de tabaco
4,A,"AGRICULTURA, GANADERÍA, CAZA, SILVICULTURA Y P...",01,"Agricultura, ganadería, caza y actividades de ...",011,Cultivos agrícolas transitorios,0115,Cultivo de plantas textiles


### Extraer Macrosector

In [9]:
ds = catalog.get("empresas_10k")
records = list(loader.load(ds))
df_10k = pd.DataFrame(records)
df_10k.macrosector.unique()

array(['MINERO', 'MANUFACTURA', 'COMERCIO', 'SERVICIOS', 'CONSTRUCCIÓN',
       'AGROPECUARIO'], dtype=object)

**Macrosector con ML**

In [10]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity
# import numpy as np

# # textos base del macrosector
# targets = {
#     "AGROPECUARIO": "agricultura ganaderia pesca silvicultura cultivo",
#     "MINERO": "minas canteras extraccion minerales petroleo gas",
#     "MANUFACTURA": "fabricacion manufactura industria procesamiento",
#     "COMERCIO": "comercio ventas mayorista minorista distribuccion",
#     "SERVICIOS": "servicios actividades profesionales transporte turismo",
#     "CONSTRUCCIÓN": "construccion edificacion obras civiles arquitectura"
# }

# # 1. vectores objetivo
# vectorizer = TfidfVectorizer()
# X_target = vectorizer.fit_transform(targets.values())

# def infer_macrosector(text):
#     vec = vectorizer.transform([text])
#     sims = cosine_similarity(vec, X_target)[0]
#     idx = sims.argmax()
#     return list(targets.keys())[idx]

# df_ciiu_desc["macrosector_ml"] = df_ciiu_desc.seccion_desc.apply(infer_macrosector)
# df_ciiu_desc.head() 
# df_sections = (
#     df_ciiu_desc[["seccion_code", "seccion_desc", "macrosector_ml"]]
#     .drop_duplicates()
#     .sort_values("seccion_code")
#     .rename(columns={"seccion_desc": "seccion"})
# )
# df_sections


**Macrosector con Heurísticas**

In [11]:
MACRO_MAP = {
    "AGROPECUARIO": ["A"],
    "MINERO": ["B"],
    "MANUFACTURA": ["C"],
    "CONSTRUCCIÓN": ["F"],
    "COMERCIO": ["G"],
    "SERVICIOS": [
        "D", "E", "H", "I", "J", "K", "L", 
        "M", "N", "O", "P", "Q", "R", "S", 
        "T", "U"
    ]
}

def infer_macrosector(section_code: str) -> str:
    if section_code is None:
        return None

    for macro, sections in MACRO_MAP.items():
        if section_code in sections:
            return macro

    return None
df_ciiu_desc["infered_macrosector"] = df_ciiu_desc["seccion_code"].apply(infer_macrosector)


### Export CIIU Dataset

In [12]:
df_ciiu_desc.to_csv(f"{DATA_PATH}/processed/ciiu.csv", index=False)
df_ciiu_desc.head(5)

,seccion_code,seccion_desc,division_code,division_desc,group_code,group_desc,ciiu_code,ciiu_desc,infered_macrosector
0,A,"AGRICULTURA, GANADERÍA, CAZA, SILVICULTURA Y P...",01,"Agricultura, ganadería, caza y actividades de ...",011,Cultivos agrícolas transitorios,0111,"Cultivo de cereales (excepto arroz), legumbres...",AGROPECUARIO
1,A,"AGRICULTURA, GANADERÍA, CAZA, SILVICULTURA Y P...",01,"Agricultura, ganadería, caza y actividades de ...",011,Cultivos agrícolas transitorios,0112,Cultivo de arroz,AGROPECUARIO
2,A,"AGRICULTURA, GANADERÍA, CAZA, SILVICULTURA Y P...",01,"Agricultura, ganadería, caza y actividades de ...",011,Cultivos agrícolas transitorios,0113,"Cultivo de hortalizas, raíces y tubérculos",AGROPECUARIO
3,A,"AGRICULTURA, GANADERÍA, CAZA, SILVICULTURA Y P...",01,"Agricultura, ganadería, caza y actividades de ...",011,Cultivos agrícolas transitorios,0114,Cultivo de tabaco,AGROPECUARIO
4,A,"AGRICULTURA, GANADERÍA, CAZA, SILVICULTURA Y P...",01,"Agricultura, ganadería, caza y actividades de ...",011,Cultivos agrícolas transitorios,0115,Cultivo de plantas textiles,AGROPECUARIO
